In [11]:
import numpy as np
from tqdm.autonotebook import tqdm
from collections import defaultdict
import json

from words import words, answers
from utilities import combinations

outcomes = combinations('gyb', 5)

words = set(words + answers)
answers = set(answers)

A = np.array([[l for l in w] for w in answers])
W = np.array([[l for l in w] for w in words])
alphabet = 'qwertyuiopasdfghjklzxcvbnm'
letter_at_index = {(i, l) : {''.join(w) for w in A[A[:, i] == l]} for l in alphabet for i in range(5)}
letter_not_at_index = {(i, l) : {''.join(w) for w in A[(A[:, i] != l)]} for l in alphabet for i in range(5)}

`allowed_by` takes in a guess, outcome, list of possible words, and utility dictionaries $\rightarrow$ returns set of words allowed by guess/outcome combination

In [12]:
def allowed_by(guess, outcome, words, at_index = letter_at_index, not_at_index = letter_not_at_index):
    
    words = words.copy()
    O = np.array([l for l in outcome])
    G = np.array([l for l in guess])
    I = np.arange(5)
    
    non_green_idxs = {l: {*I[~((O == 'g') & (G == l))]} for l in G}
        
    for idx in range(5):
        l = guess[idx]
        o = outcome[idx]
        
        # filter for greens first
        if o == 'g':
            words = words.intersection(at_index[(idx, l)])
            
            if len(words) == 0:
                return words
            
        # then yellows
        elif o == 'y':
            
            # retain all words without the yellow letter at its spot
            words = words.intersection(not_at_index[(idx, l)])
            
            # retain all words with the yellow letter but not in the green spots
            in_word = set()
            for j in (non_green_idxs[l] - {idx}):
                in_word = in_word.union(at_index[(j, l)])
                
            words = words.intersection(in_word)
            
            if len(words) == 0:
                return words
            
        # then blacks
        elif o == 'b':
                        
            # retain all words without the letter in any non-green spot
            for j in non_green_idxs[l]:
                words = words.intersection(not_at_index[(j, l)])
                
                if len(words) == 0:
                    return words
            
        else:
            raise Exception(f'outcome can only include g, y, b, but is {outcome}')
            
    return words

For every guess/outcome combination, save the resultant set of possible answers if its measure is > 0

In [13]:
def save_first_level(file):

    first_level = {(g, outcome) : allowed_by(g, outcome, answers) for g in tqdm(words) for outcome in outcomes}

    with open(file, 'w') as file:
        json.dump({'/'.join(k): [*v] for k, v in first_level.items() if len(v) > 0}, file)
        
    entropies = {}

    for k, v in first_level.items():

        word = k[:5]

        if word not in entropies:
            entropies[word] = 0

        n = len(v)

        p = n / len(answers)

        entropies[word] -= p * np.log2(p)

    with open('first_level_entropies.json', 'w') as file:
        json.dump(entropies, file)

Load the results of the previous computation

In [17]:
with open('first_level.json', 'r') as file:
    first_level = {k : set(v) for k, v in json.load(file)}

In [24]:
with open('first_level_entropies.json', 'r') as file:
    entropies = json.load(file)

Select the ten highest entropy words as finalists

In [15]:
finalists = sorted(entropies, key = entropies.get)[::-1][:10]
keys = [k for k in first_level if any(f in k for f in finalists)]
keys = sorted(keys, key = lambda x: len(first_level[x]))

print('Finalists:'+'\n\t'.join(finalists))

In [18]:
second_level = {}

for k1 in tqdm(keys):
    g1 = np.array(list(k1.split('/')[0]))
    o1 = np.array(list(k1.split('/')[1]))
    
    remaining = first_level[k1]
    
    if ''.join(o1) == 'ggggg':
        second_level.update({k1:None})
    
    if len(remaining) <= 2:
        for r in remaining:
            second_level.update({k1 + f'//{r}/ggggg':None})
        continue
    
    for outcome in outcomes:
        
        # Make a list of guesses consistent with this outcome given prior information
        G = A.copy()
        o2 = np.array(list(outcome))
        
        # Where both outcomes are green, guesses must have same letter
        both_green = (o1 == 'g') & (o2 == 'g')
        if sum(both_green) > 0:
            G = G[(G[:, both_green] == g1[both_green]).min(1)]
            
        # Where both guesses are the same, outcomes must be the same
        G = G[((G != g1) | (o2 == o1)).min(1)]
        
        # Note: there are more conditions one could add but not clear how much of a speed-up they obtain?
        
        for g2 in G:
            guess = ''.join(g2)
            k2 = f'{guess}/{outcome}'
            
            if k2 in first_level:
                key = k1 + '//' + k2
                result = first_level[k2].intersection(remaining)
                if len(result) > 0:
                    second_level[key] = [*result]

In [20]:
with open('second_level.json', 'w') as file:
    json.dump(second_level, file)

Todo: find best first word using two-level search

In [ ]:
best_guess_2d = {}

for k, v in second_level.items():
    
    first, second = k.split('//')
    
    n = len(v)

In [32]:
entropies2 = {}

for k, v in tqdm(second_level.items()):
    
    word = k.split('/')[0]
    
    if word not in entropies2:
        entropies2[word] = 0
    
    if v is not None:
        n = len(v)
    
        p = n / len(answers)
    
        entropies2[word] -= p * np.log2(p)

In [23]:
entropies2 = entropies.copy()

In [33]:
import pandas as pd
pd.DataFrame([entropies, entropies2]).T.reindex(entropies2).sort_values(0)

,0,1
trace,5.830549,17631.485169
irate,5.831397,17674.686747
salet,5.834582,17717.598171
crate,5.834874,17662.149248
slate,5.855775,17730.111401
reast,5.865457,17718.300239
raile,5.865710,17830.984431
raise,5.877910,17782.917604
roate,5.882779,17839.056084
soare,5.885960,17868.109780
